# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-b430e0c149-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Tennis explorative search") 

Consider the following exploratory information need:

> investigate tennis and find the main BGPs related to this sport. Compare players per countries, awards of the players and tournament.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q1426`| Roger Federer       | node |
| `wd:Q11459`| Serena Williams       | node |





Also consider

```
wd:Q1426 ?p ?obj .
```

is the BGP to retrieve all **properties of Roger Federer**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for tennis player

2. Identify the BGP for tennis tournament

3. Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

4. Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

5. Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

6. Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

7. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

8. Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

## Task 1
Identify the BGP for tennis player

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q1426 ?p ?obj . #Federer
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('name', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('name', 'Munzinger person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('name', 

In [3]:
#We need to find the BGP of tennis player and we already have the one of roger federer. 
#Let's check what Roger Federer is instance of

queryString = """
SELECT DISTINCT ?type ?Tname
WHERE {
   wd:Q1426 wdt:P31 ?type . #Federer a
   
   ?type sc:name ?Tname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('type', 'http://www.wikidata.org/entity/Q5'), ('Tname', 'human')]
1


In [4]:
#He is not instance of tennis player, let's check his occupations (we got the property within the first query result)

queryString = """
SELECT DISTINCT ?occupation ?Oname
WHERE {
   wd:Q1426 wdt:P106 ?occupation . #Federer occupation
   
   ?occupation sc:name ?Oname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q10833314'), ('Oname', 'tennis player')]
1


Final query for this task

In [6]:
# Final query

queryString = """
SELECT DISTINCT ?occupation ?Oname
WHERE {
   wd:Q1426 wdt:P106 ?occupation . #Federer occupation
   
   ?occupation sc:name ?Oname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

#[('occupation', 'http://www.wikidata.org/entity/Q10833314'), ('Oname', 'tennis player')]

Results
[('occupation', 'http://www.wikidata.org/entity/Q10833314'), ('Oname', 'tennis player')]
1


## Task 2
Identify the BGP for tennis tournament

In [7]:
#In the first query, we got the "participated in" property, let's explore it

queryString = """
SELECT DISTINCT ?participation ?Pname
WHERE {
   wd:Q1426 wdt:P1344 ?participation . #Federer participantIn
   
   ?participation sc:name ?Pname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('participation', 'http://www.wikidata.org/entity/Q589310'), ('Pname', "tennis at the 2000 Summer Olympics – men's singles")]
[('participation', 'http://www.wikidata.org/entity/Q8544'), ('Pname', '2000 Summer Olympics')]
[('participation', 'http://www.wikidata.org/entity/Q8558'), ('Pname', '2004 Summer Olympics')]
[('participation', 'http://www.wikidata.org/entity/Q581788'), ('Pname', "tennis at the 2012 Summer Olympics – men's singles")]
[('participation', 'http://www.wikidata.org/entity/Q757054'), ('Pname', "tennis at the 2008 Summer Olympics – men's doubles")]
5


In [8]:
#Let's take one of the results and check their type, which could be tennis tournament

queryString = """
SELECT DISTINCT ?participationType ?Pname
WHERE {
   wd:Q1426 wdt:P1344 ?participation . #Federer participantIn
   ?participation wdt:P31 ?participationType. #a
   
   ?participationType sc:name ?Pname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('participationType', 'http://www.wikidata.org/entity/Q159821'), ('Pname', 'Summer Olympic Games')]
[('participationType', 'http://www.wikidata.org/entity/Q18536594'), ('Pname', 'Olympic sporting event')]
[('participationType', 'http://www.wikidata.org/entity/Q46190676'), ('Pname', 'tennis event')]
3


In [16]:
#Intuitively, "tennis event" looks interesting, let's see what it is instance of

queryString = """
SELECT DISTINCT ?obj ?Oname
WHERE {

   wd:Q46190676 wdt:P31 ?obj .

   ?obj sc:name ?Oname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [20]:
#Let's check its subclasses 

queryString = """
SELECT DISTINCT ?tennisThing ?Tname
WHERE {

   ?tennisThing wdt:P279 wd:Q46190676. #subclassOf tennisEvent 

   ?tennisThing sc:name ?Tname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('tennisThing', 'http://www.wikidata.org/entity/Q21170330'), ('Tname', 'tennis qualification event')]
[('tennisThing', 'http://www.wikidata.org/entity/Q50843310'), ('Tname', 'tennis team event')]
2


In [17]:
#Let's see all the related prop/obj to tennis event

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname 
WHERE {

   wd:Q46190676 ?p ?obj . #tennisEvent

   ?obj sc:name ?Oname.
   ?p sc:name ?Pname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('Pname', 'has quality'), ('obj', 'http://www.wikidata.org/entity/Q1967745'), ('Oname', 'type of tennis match')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P710'), ('Oname', 'participant')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P641'), ('Oname', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P361'), ('Oname', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P2094'), ('Oname', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1346'), ('Oname', 'winner')]
[('

In [18]:
#Same query as above but for the "tennis tournament edition" class

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname 
WHERE {

   wd:Q47345468 ?p ?obj . #tennisTournamentEdition

   ?obj sc:name ?Oname.
   ?p sc:name ?Pname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P527'), ('Oname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P276'), ('Oname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P582'), ('Oname', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P641'), ('Oname', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P393'), ('Oname', 'edition number')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P580'), ('Oname', 'start time')]
[('p', 'h

Final query for this task

In [62]:
# Final query

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname 
WHERE {

   wd:Q47345468 ?p ?obj . #tennisTournamentEdition

   ?obj sc:name ?Oname.
   ?p sc:name ?Pname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

#('obj', 'http://www.wikidata.org/entity/Q13219666'), ('Oname', 'tennis tournament')]

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P527'), ('Oname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P276'), ('Oname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P582'), ('Oname', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P641'), ('Oname', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P393'), ('Oname', 'edition number')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P580'), ('Oname', 'start time')]
[('p', 'h

## Task 3
Return the four Grand Slam of tennis (hint: One of these four is Wimbledon) (the result set must be a list of four couples Grand Slam IRI and label).

In [7]:
# We previously got the bgp of tennis tournament, and we are given here the name of one of the grand slams, let's try to exploit it with regex and see instances

queryString = """
SELECT DISTINCT ?tournament ?Tname 
WHERE {

   ?tournament wdt:P31 wd:Q13219666. #tennisTournament a
   
   ?tournament sc:name ?Tname.
   FILTER(REGEX(?Tname, "[Ww]imbledon"))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [8]:
#Previous query but larger approach (including all prop)

queryString = """
SELECT DISTINCT ?tournament ?Tname 
WHERE {

   ?tournament ?p wd:Q13219666. #tennisTournament a
   
   ?tournament sc:name ?Tname.
   FILTER(REGEX(?Tname, "[Ww]imbledon"))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('tournament', 'http://www.wikidata.org/entity/Q41520'), ('Tname', 'Wimbledon Championships')]
[('tournament', 'http://www.wikidata.org/entity/Q12776'), ('Tname', 'Wimbledon Plate')]
2


In [11]:
#Let's see what "Wimbledon Championship" is instance of 

queryString = """
SELECT DISTINCT ?type ?Tname
WHERE {
   wd:Q41520 wdt:P31 ?type . #WimbledonChampionships a
   
   ?type sc:name ?Tname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('type', 'http://www.wikidata.org/entity/Q18608583'), ('Tname', 'recurring sporting event')]
1


In [12]:
#Let's see related obj/prop

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname 
WHERE {

   wd:Q41520 ?p ?obj . #WimbledonChampionships

   ?obj sc:name ?Oname.
   ?p sc:name ?Pname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

#We found the bgp of Grand slam, related to Wimbledon through the property "part of"

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('Pname', 'located in the administrative territorial entity'), ('obj', 'http://www.wikidata.org/entity/Q736742'), ('Oname', 'Wimbledon')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('Pname', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q43323825'), ('Oname', 'Template:Wimbledon player')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('Pname', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P4503'), ('Oname', 'Wimbledon player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('Pname', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('Oname', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('Pname', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q206375'), ('Oname', 'Wimbledon')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('Pname', 'has list'), ('obj', 'http://www.wikidata.org/entity/Q6585893'), ('Oname', 'list o

In [65]:
#We found the bgp of Grand slam, related to Wimbledon through the property "part of"
#Let's see if we can get the 3 others the same way

queryString = """
SELECT DISTINCT ?slam ?slamName
WHERE {

   ?slam wdt:P361 wd:Q102113 ; #partOf GranSlams
         sc:name ?slamName
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('slam', 'http://www.wikidata.org/entity/Q43605'), ('slamName', 'French Open')]
[('slam', 'http://www.wikidata.org/entity/Q123577'), ('slamName', 'US Open')]
[('slam', 'http://www.wikidata.org/entity/Q41520'), ('slamName', 'Wimbledon Championships')]
[('slam', 'http://www.wikidata.org/entity/Q60874'), ('slamName', 'Australian Open')]
4


Final query for this task

In [66]:
# Final query

queryString = """
SELECT DISTINCT ?slam ?slamName
WHERE {

   ?slam wdt:P361 wd:Q102113 ; #partOf GranSlams
         sc:name ?slamName
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('slam', 'http://www.wikidata.org/entity/Q43605'), ('slamName', 'French Open')]
[('slam', 'http://www.wikidata.org/entity/Q123577'), ('slamName', 'US Open')]
[('slam', 'http://www.wikidata.org/entity/Q41520'), ('slamName', 'Wimbledon Championships')]
[('slam', 'http://www.wikidata.org/entity/Q60874'), ('slamName', 'Australian Open')]
4


## Task 4
Find the three different type of possible surfaces in a tennis court (the result set must be a list of three couples surface IRI and label).

In [15]:
# We can re-use the result of the query above :
#[('p', 'http://www.wikidata.org/prop/direct/P765'), ('Pname', 'surface played on'), ('obj', 'http://www.wikidata.org/entity/Q1151291'), ('Oname', 'grass court')]

#Let's check subjects and objects related to the property "surface played on"


queryString = """
SELECT DISTINCT ?Sname ?Oname 
WHERE {

   ?subj wdt:P765 ?obj.
   
   ?subj sc:name ?Sname.
   ?obj sc:name ?Oname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('Sname', 'Virginia Slims of Detroit'), ('Oname', 'carpet')]
[('Sname', '2000 Open Gaz de France'), ('Oname', 'carpet')]
[('Sname', '2000 Toray Pan Pacific Open'), ('Oname', 'carpet')]
[('Sname', '2003 Toray Pan Pacific Open'), ('Oname', 'carpet')]
[('Sname', '1992 Volkswagen Cup Damen Grand Prix'), ('Oname', 'carpet')]
[('Sname', '1993 Volkswagen Card Cup'), ('Oname', 'carpet')]
[('Sname', '2001 Sparkassen Cup'), ('Oname', 'carpet')]
[('Sname', '2003 Sparkassen Cup'), ('Oname', 'carpet')]
[('Sname', '2002 Sparkassen Cup'), ('Oname', 'carpet')]
[('Sname', '2001 Toray Pan Pacific Open'), ('Oname', 'carpet')]
[('Sname', '1996 Sparkassen Cup'), ('Oname', 'carpet')]
[('Sname', 'Sparkassen Cup'), ('Oname', 'carpet')]
[('Sname', '1991 Volkswagen Cup'), ('Oname', 'carpet')]
[('Sname', '2014 Rally Azores'), ('Oname', 'gravel')]
[('Sname', '2013 Rally Australia'), ('Oname', 'gravel')]
[('Sname', '2014 Rally Australia'), ('Oname', 'gravel')]
[('Sname', '2015 Rally Australia'), ('Oname',

In [18]:
#Let's change approach and use the "grass court" bgp 
#Related obj and prop

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname 
WHERE {

   wd:Q1151291 ?p ?obj . #GrassCourt

   ?obj sc:name ?Oname.
   ?p sc:name ?Pname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('Pname', 'uses'), ('obj', 'http://www.wikidata.org/entity/Q47678177'), ('Oname', 'grass')]
[('p', 'http://www.wikidata.org/prop/direct/P2283'), ('Pname', 'uses'), ('obj', 'http://www.wikidata.org/entity/Q47769255'), ('Oname', 'outdoor grass')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('Pname', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q47677460'), ('Oname', 'tennis playing surface')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('Pname', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q847'), ('Oname', 'tennis')]
4


In [24]:
#Previous query shows that grass court is subclassOf "tennis playing surface", let's see all the subclasses :

queryString = """
SELECT DISTINCT ?surface ?Sname
WHERE {

   ?surface wdt:P279 wd:Q47677460. #subclassOf tennisPlayingSurface 

   ?surface sc:name ?Sname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

#The question asks for 3, not 19, let's assume we want the 3 "main ones" from the Gran Slams 

Results
[('surface', 'http://www.wikidata.org/entity/Q47677214'), ('Sname', 'carpet')]
[('surface', 'http://www.wikidata.org/entity/Q47676928'), ('Sname', 'clay')]
[('surface', 'http://www.wikidata.org/entity/Q1151291'), ('Sname', 'grass court')]
[('surface', 'http://www.wikidata.org/entity/Q47678177'), ('Sname', 'grass')]
[('surface', 'http://www.wikidata.org/entity/Q47769232'), ('Sname', 'indoor hard')]
[('surface', 'http://www.wikidata.org/entity/Q47769271'), ('Sname', 'outdoor hard')]
[('surface', 'http://www.wikidata.org/entity/Q47777555'), ('Sname', 'hardcourt surface')]
[('surface', 'http://www.wikidata.org/entity/Q47769216'), ('Sname', 'indoor carpet')]
[('surface', 'http://www.wikidata.org/entity/Q47769202'), ('Sname', 'indoor')]
[('surface', 'http://www.wikidata.org/entity/Q47769177'), ('Sname', 'asphalt')]
[('surface', 'http://www.wikidata.org/entity/Q47769149'), ('Sname', 'acrylic')]
[('surface', 'http://www.wikidata.org/entity/Q47769166'), ('Sname', 'artificial grass')]
[(

In [31]:
#The question asks for 3, not 19, let's assume we want the 3 "main ones" from the Gran Slams 
#To do so, let's reuse "surfacePlayedOn" but only for gran slams

queryString = """
SELECT DISTINCT ?Slamname ?Surfname 
WHERE {
   ?Slam ?p wd:Q102113; #GranSlam
         wdt:P765 ?surface. #surfacePlayedOn
   
    ?surface sc:name ?Surfname.
    ?Slam sc:name ?Slamname

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('Slamname', 'French Open'), ('Surfname', 'clay court')]
[('Slamname', 'US Open'), ('Surfname', 'hardcourt')]
[('Slamname', 'Wimbledon Championships'), ('Surfname', 'grass court')]
[('Slamname', 'Australian Open'), ('Surfname', 'hardcourt')]
[('Slamname', 'Australian Open'), ('Surfname', 'grass court')]
5


In [32]:
#Let's adjust the result s format

queryString = """
SELECT DISTINCT ?surface ?Sname 
WHERE {
   ?Slam ?p wd:Q102113; #GranSlam
         wdt:P765 ?surface. #surfacePlayedOn
   
    ?surface sc:name ?Sname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('surface', 'http://www.wikidata.org/entity/Q198718'), ('Sname', 'clay court')]
[('surface', 'http://www.wikidata.org/entity/Q1330871'), ('Sname', 'hardcourt')]
[('surface', 'http://www.wikidata.org/entity/Q1151291'), ('Sname', 'grass court')]
3


Final query for this task

In [33]:
# Final query

queryString = """
SELECT DISTINCT ?surface ?Sname 
WHERE {
   ?Slam ?p wd:Q102113; #GranSlam
         wdt:P765 ?surface. #surfacePlayedOn
   
    ?surface sc:name ?Sname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('surface', 'http://www.wikidata.org/entity/Q198718'), ('Sname', 'clay court')]
[('surface', 'http://www.wikidata.org/entity/Q1330871'), ('Sname', 'hardcourt')]
[('surface', 'http://www.wikidata.org/entity/Q1151291'), ('Sname', 'grass court')]
3


## Task 5
Return the tournaments played in the 2018 Wimbledon Championships edition with the related winners (hint: men and women do not play together, there are two distinct tournament ) (the result set must be a list of elements with tournament IRI and label, and winner IRI and label).

In [41]:
# A previous query result was : [('tournament', 'http://www.wikidata.org/entity/Q41520'), ('Tname', 'Wimbledon Championships')]
#Let's see if the year is a data property 

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname 
WHERE {

   wd:Q41520 ?p ?obj . #Wimbledon

   ?obj sc:name ?Oname.
   ?p sc:name ?Pname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('Pname', 'located in the administrative territorial entity'), ('obj', 'http://www.wikidata.org/entity/Q736742'), ('Oname', 'Wimbledon')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('Pname', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q43323825'), ('Oname', 'Template:Wimbledon player')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('Pname', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P4503'), ('Oname', 'Wimbledon player ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('Pname', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('Oname', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('Pname', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q206375'), ('Oname', 'Wimbledon')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('Pname', 'has list'), ('obj', 'http://www.wikidata.org/entity/Q6585893'), ('Oname', 'list o

In [40]:
#Let's check now instances of Wimbledon

queryString = """
SELECT DISTINCT ?instance ?Iname
WHERE {

   ?instance wdt:P31 wd:Q41520. #a WimbledonChampionship

   ?instance sc:name ?Iname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q268871'), ('Iname', '2004 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q173961'), ('Iname', '1978 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q925026'), ('Iname', '1971 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q15854765'), ('Iname', '2014 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q30085309'), ('Iname', '2018 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q745901'), ('Iname', '1974 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q655368'), ('Iname', '1955 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q655378'), ('Iname', '1956 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q657950'), ('Iname', '1957 Wimbledon Championships')]
[('instance', 'http://www.wikidata.org/entity/Q657962'), ('Iname', '1958 Wimbledon Champ

In [45]:
#Let's get the 2018 Wim Champ bgp

queryString = """
SELECT DISTINCT ?instance ?Iname
WHERE {

   ?instance wdt:P31 wd:Q41520. #a WimbledonChampionship

   ?instance sc:name ?Iname.
   FILTER(REGEX(?Iname,"2018"))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q30085309'), ('Iname', '2018 Wimbledon Championships')]
1


In [3]:
#Let's explore now related obj/prop

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname 
WHERE {

   wd:Q30085309 ?p ?obj . #2018Wimbledon

   ?obj sc:name ?Oname.
   ?p sc:name ?Pname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('Pname', 'located in the administrative territorial entity'), ('obj', 'http://www.wikidata.org/entity/Q736742'), ('Oname', 'Wimbledon')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('Pname', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q25344121'), ('Oname', '2017 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('Pname', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q54093953'), ('Oname', '2019 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('Pname', 'country'), ('obj', 'http://www.wikidata.org/entity/Q145'), ('Oname', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('Pname', 'location'), ('obj', 'http://www.wikidata.org/entity/Q84'), ('Oname', 'London')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('Pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q41520'), ('Oname', 'Wimbledon Championships')]


In [4]:
#We see that the property "hasPart" links the main event to all its "subtournament", which are what we are looking for
#Now that we have the tournaments, we must find how to get the winners, let's take one subtournament randomly and explore related obj/prop

#Let's explore now related obj/prop

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname 
WHERE {

   wd:Q30098268 ?p ?obj . #2018WimbledonWomenSingles

   ?obj sc:name ?Oname.
   ?p sc:name ?Pname
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('Pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q77178'), ('Oname', 'Angelique Kerber')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('Pname', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q25344122'), ('Oname', "2017 Wimbledon Championships – Women's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('Pname', 'followed by'), ('obj', 'http://www.wikidata.org/entity/Q55749169'), ('Oname', "2019 Wimbledon Championships – Women's Singles")]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('Pname', 'competition class'), ('obj', 'http://www.wikidata.org/entity/Q16893403'), ('Oname', "women's singles")]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('Pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q46190676'), ('Oname', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('Pname', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q30085309'), ('Ona

In [10]:
#We see that there is indeed a property winner, so we can start building the query :

queryString = """
SELECT DISTINCT ?tournament ?Tname ?winner ?Wname 
WHERE {

   wd:Q30085309 wdt:P527 ?tournament . #2018Wimbledon hasPart
   ?tournament wdt:P1346 ?winner. #winner
   
   ?tournament sc:name ?Tname.
   ?winner sc:name ?Wname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('tournament', 'http://www.wikidata.org/entity/Q32357790'), ('Tname', "2018 Wimbledon Championships – Men's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q53342'), ('Wname', 'Mike Bryan')]
[('tournament', 'http://www.wikidata.org/entity/Q54961494'), ('Tname', "2018 Wimbledon Championships – Men's Singles"), ('winner', 'http://www.wikidata.org/entity/Q5812'), ('Wname', 'Novak Djokovic')]
[('tournament', 'http://www.wikidata.org/entity/Q30098268'), ('Tname', "2018 Wimbledon Championships – Women's Singles"), ('winner', 'http://www.wikidata.org/entity/Q77178'), ('Wname', 'Angelique Kerber')]
[('tournament', 'http://www.wikidata.org/entity/Q32356704'), ('Tname', "2018 Wimbledon Championships – Women's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q11052500'), ('Wname', 'Barbora Krejčíková')]
[('tournament', 'http://www.wikidata.org/entity/Q55455423'), ('Tname', "2018 Wimbledon Championships – Wheelchair Women's Doubles"), ('winner', 'http://www.wikidata.org/entity/

In [13]:
#Men's double tournament only returns one person, but it should return 2, let's double check 2018 men's doubles winner.

queryString = """
SELECT DISTINCT ?winner ?Wname 
WHERE {

   wd:Q32357790 wdt:P1346 ?winner. #2018WibledonMensDouble winner

    ?winner sc:name ?Wname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q53342'), ('Wname', 'Mike Bryan')]
1


In [28]:
#Let's see all related props/obj

queryString = """
SELECT DISTINCT ?p ?Pname ?s ?Sname 
WHERE {

   wd:Q32357790 ?p ?s. #2018WibledonMensDouble

    ?p sc:name ?Pname.
    ?s sc:name ?Sname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('Pname', 'part of'), ('s', 'http://www.wikidata.org/entity/Q30085309'), ('Sname', '2018 Wimbledon Championships')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('Pname', 'country'), ('s', 'http://www.wikidata.org/entity/Q145'), ('Sname', 'United Kingdom')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('Pname', 'sport'), ('s', 'http://www.wikidata.org/entity/Q847'), ('Sname', 'tennis')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('Pname', 'winner'), ('s', 'http://www.wikidata.org/entity/Q53342'), ('Sname', 'Mike Bryan')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('Pname', 'instance of'), ('s', 'http://www.wikidata.org/entity/Q46190676'), ('Sname', 'tennis event')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('Pname', 'competition class'), ('s', 'http://www.wikidata.org/entity/Q17299700'), ('Sname', "men's doubles")]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('Pname', 'followed

In [39]:
#Let's see if there is a winner subproperty to get the other one 

queryString = """
SELECT DISTINCT ?p ?Pname 
WHERE {
    wd:P1346 wdt:P1647 ?p. #Winner subproperty
    
    ?p sc:name ?Pname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/P710'), ('Pname', 'participant')]
1


Final query for this task

In [59]:
# Final query

queryString = """
SELECT DISTINCT ?tournament ?Tname ?winner ?Wname 
WHERE {

   wd:Q30085309 wdt:P527 ?tournament . #2018Wimbledon hasPart
   ?tournament wdt:P1346 ?winner. #winner
   
   ?tournament sc:name ?Tname.
   ?winner sc:name ?Wname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

#Men's double has only one winner displayed here

Results
[('tournament', 'http://www.wikidata.org/entity/Q32357790'), ('Tname', "2018 Wimbledon Championships – Men's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q53342'), ('Wname', 'Mike Bryan')]
[('tournament', 'http://www.wikidata.org/entity/Q54961494'), ('Tname', "2018 Wimbledon Championships – Men's Singles"), ('winner', 'http://www.wikidata.org/entity/Q5812'), ('Wname', 'Novak Djokovic')]
[('tournament', 'http://www.wikidata.org/entity/Q30098268'), ('Tname', "2018 Wimbledon Championships – Women's Singles"), ('winner', 'http://www.wikidata.org/entity/Q77178'), ('Wname', 'Angelique Kerber')]
[('tournament', 'http://www.wikidata.org/entity/Q32356704'), ('Tname', "2018 Wimbledon Championships – Women's Doubles"), ('winner', 'http://www.wikidata.org/entity/Q11052500'), ('Wname', 'Barbora Krejčíková')]
[('tournament', 'http://www.wikidata.org/entity/Q55455423'), ('Tname', "2018 Wimbledon Championships – Wheelchair Women's Doubles"), ('winner', 'http://www.wikidata.org/entity/

## Task 6
Consider the tennis players born after 1975, return who won more Grand Slams and cut the result to the top-10 (the result set must be a list of triples player IRI, label and #victories).

In [61]:
# For this query, we need bgp of Grand slam (hasPart and partOf), tennis player, winner, birth date. The only one missing is birth date, let's find it
#Let's also note that we assume the question refers to all tournaments of Gran Slams (so we include Doubles and so on)

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname
WHERE {
   wd:Q5 ?p ?obj. #Human

   ?p sc:name ?Pname.
   ?obj sc:name ?Oname.
   FILTER(REGEX(?Oname,"(birth|born|date)"))

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

#we got date of birth P569

Results
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P570'), ('Oname', 'date of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P19'), ('Oname', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1636'), ('Oname', 'date of baptism in early childhood')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1477'), ('Oname', 'birth name')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P4602'), ('Oname', 'date of burial or cremation')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikid

In [78]:
#tennis player : Q10833314
#gran slam : Q102113
#winner : P1346 bdate: P569

#Let's try to build the query (bgp for profession is given), first checking if birth dates are returned

queryString = """
SELECT DISTINCT ?bdate
WHERE {
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P569 ?bdate.          #birthdate

}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('bdate', '1957-07-24T00:00:00Z')]
[('bdate', '1994-09-13T00:00:00Z')]
[('bdate', '1989-04-17T00:00:00Z')]
[('bdate', '1941-09-13T00:00:00Z')]
[('bdate', '1930-01-01T00:00:00Z')]
[('bdate', '1995-07-08T00:00:00Z')]
[('bdate', '1981-09-19T00:00:00Z')]
[('bdate', '1994-09-01T00:00:00Z')]
[('bdate', '1928-02-02T00:00:00Z')]
[('bdate', '1899-11-04T00:00:00Z')]
10


In [80]:
queryString = """
SELECT DISTINCT ?player ?Pname
WHERE {
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P569 ?bdate.          #birthdate
           
   ?tournament wdt:P31 wd:Q102113; #a GranSlam
               wdt:P1346 ?player. #winner
   
   ?player sc:name ?Pname.

}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [93]:
#We are not accessing slams right. Let's try do display them all :

queryString = """
SELECT DISTINCT ?tournament ?TName
WHERE {
           
    ?tournament wdt:P31 ?granSlam. #a
    ?granSlam wdt:P361 wd:Q102113. #partOf GranSlams
    
    ?tournament sc:name ?TName

}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('tournament', 'http://www.wikidata.org/entity/Q268871'), ('TName', '2004 Wimbledon Championships')]
[('tournament', 'http://www.wikidata.org/entity/Q602116'), ('TName', '1969 Australian Open')]
[('tournament', 'http://www.wikidata.org/entity/Q643498'), ('TName', '2000 US Open')]
[('tournament', 'http://www.wikidata.org/entity/Q660527'), ('TName', '1983 French Open')]
[('tournament', 'http://www.wikidata.org/entity/Q840907'), ('TName', '2004 US Open')]
[('tournament', 'http://www.wikidata.org/entity/Q846695'), ('TName', '2000 French Open')]
[('tournament', 'http://www.wikidata.org/entity/Q173961'), ('TName', '1978 Wimbledon Championships')]
[('tournament', 'http://www.wikidata.org/entity/Q925026'), ('TName', '1971 Wimbledon Championships')]
[('tournament', 'http://www.wikidata.org/entity/Q776766'), ('TName', '1928 French Championships')]
[('tournament', 'http://www.wikidata.org/entity/Q15854765'), ('TName', '2014 Wimbledon Championships')]
10


In [96]:
#And if we want to access the different tournaments of all slams :

queryString = """
SELECT DISTINCT ?parts ?PName
WHERE {
    
    ?granSlam wdt:P361 wd:Q102113. #partOf GranSlams
    ?tournament wdt:P31 ?granSlam; #a
                wdt:P527 ?parts. #hasPart
    ?parts sc:name ?PName

}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('parts', 'http://www.wikidata.org/entity/Q15854755'), ('PName', "1978 Wimbledon Championships – Men's Doubles")]
[('parts', 'http://www.wikidata.org/entity/Q20895284'), ('PName', '2015 US Open – Mixed Doubles')]
[('parts', 'http://www.wikidata.org/entity/Q15854758'), ('PName', "1982 Wimbledon Championships – Men's Doubles")]
[('parts', 'http://www.wikidata.org/entity/Q22231236'), ('PName', "2016 Australian Open – Boys' Doubles")]
[('parts', 'http://www.wikidata.org/entity/Q22956224'), ('PName', "2017 Australian Open – Men's Singles")]
[('parts', 'http://www.wikidata.org/entity/Q25344122'), ('PName', "2017 Wimbledon Championships – Women's Singles")]
[('parts', 'http://www.wikidata.org/entity/Q30098268'), ('PName', "2018 Wimbledon Championships – Women's Singles")]
[('parts', 'http://www.wikidata.org/entity/Q1455356'), ('PName', "1980 French Open – Women's Doubles")]
[('parts', 'http://www.wikidata.org/entity/Q1751593'), ('PName', "1958 Australian Championships – Men's Singles

In [98]:
#Let's get back to the query and assemble everything and display gran slams winners :

queryString = """
SELECT DISTINCT ?player ?Pname
WHERE {
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P569 ?bdate.          #birthdate
           
    ?granSlam wdt:P361 wd:Q102113. #partOf GranSlams
    ?tournament wdt:P31 ?granSlam; #a
                wdt:P527 ?parts. #hasPart
                
    ?parts wdt:P1346 ?player.
    ?player sc:name ?Pname.

}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q299959'), ('Pname', 'Eileen Bennett Whittingstall')]
[('player', 'http://www.wikidata.org/entity/Q18918709'), ('Pname', 'Markéta Vondroušová')]
[('player', 'http://www.wikidata.org/entity/Q193661'), ('Pname', 'Stan Wawrinka')]
[('player', 'http://www.wikidata.org/entity/Q54546'), ('Pname', 'Nancy Richey')]
[('player', 'http://www.wikidata.org/entity/Q14905846'), ('Pname', 'Borna Ćorić')]
[('player', 'http://www.wikidata.org/entity/Q207352'), ('Pname', 'Agnieszka Radwańska')]
[('player', 'http://www.wikidata.org/entity/Q22958938'), ('Pname', 'Alex de Minaur')]
[('player', 'http://www.wikidata.org/entity/Q22959065'), ('Pname', 'Blake Ellis')]
[('player', 'http://www.wikidata.org/entity/Q239758'), ('Pname', 'Arantxa Rus')]
[('player', 'http://www.wikidata.org/entity/Q435255'), ('Pname', 'Shirley Brasher')]
10


In [101]:
queryString = """
SELECT DISTINCT ?player ?Pname  (count(*) as ?countWin)
WHERE {
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P569 ?bdate.          #birthdate
           
    ?granSlam wdt:P361 wd:Q102113. #partOf GranSlams
    ?tournament wdt:P31 ?granSlam; #a
                wdt:P527 ?parts. #hasPart
                
    ?parts wdt:P1346 ?player. #winner
    ?player sc:name ?Pname.
    
    FILTER(?bdate > "1975"^^xsd:gYear)

}
GROUP BY ?player ?Pname 
ORDER BY DESC(?countWin)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q11459'), ('Pname', 'Serena Williams'), ('countWin', '36')]
[('player', 'http://www.wikidata.org/entity/Q134720'), ('Pname', 'Martina Hingis'), ('countWin', '23')]
[('player', 'http://www.wikidata.org/entity/Q11578'), ('Pname', 'Venus Williams'), ('countWin', '19')]
[('player', 'http://www.wikidata.org/entity/Q10132'), ('Pname', 'Rafael Nadal'), ('countWin', '19')]
[('player', 'http://www.wikidata.org/entity/Q5812'), ('Pname', 'Novak Djokovic'), ('countWin', '19')]
[('player', 'http://www.wikidata.org/entity/Q1426'), ('Pname', 'Roger Federer'), ('countWin', '17')]
[('player', 'http://www.wikidata.org/entity/Q53340'), ('Pname', 'Bob Bryan'), ('countWin', '14')]
[('player', 'http://www.wikidata.org/entity/Q53342'), ('Pname', 'Mike Bryan'), ('countWin', '11')]
[('player', 'http://www.wikidata.org/entity/Q229128'), ('Pname', 'Cara Black'), ('countWin', '10')]
[('player', 'http://www.wikidata.org/entity/Q54537'), ('Pname', 'Bethanie Mattek

Final query for this task

In [2]:
# Final query :

queryString = """
SELECT DISTINCT ?player ?Pname  (count(*) as ?countWin)
WHERE {
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P569 ?bdate.          #birthdate
           
    ?granSlam wdt:P361 wd:Q102113. #partOf GranSlams
    ?tournament wdt:P31 ?granSlam; #a
                wdt:P527 ?parts. #hasPart
                
    ?parts wdt:P1346 ?player. #winner
    ?player sc:name ?Pname.
    
    FILTER(?bdate > "1975"^^xsd:gYear)

}
GROUP BY ?player ?Pname 
ORDER BY DESC(?countWin)
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('player', 'http://www.wikidata.org/entity/Q11459'), ('Pname', 'Serena Williams'), ('countWin', '36')]
[('player', 'http://www.wikidata.org/entity/Q134720'), ('Pname', 'Martina Hingis'), ('countWin', '23')]
[('player', 'http://www.wikidata.org/entity/Q11578'), ('Pname', 'Venus Williams'), ('countWin', '19')]
[('player', 'http://www.wikidata.org/entity/Q10132'), ('Pname', 'Rafael Nadal'), ('countWin', '19')]
[('player', 'http://www.wikidata.org/entity/Q5812'), ('Pname', 'Novak Djokovic'), ('countWin', '19')]
[('player', 'http://www.wikidata.org/entity/Q1426'), ('Pname', 'Roger Federer'), ('countWin', '17')]
[('player', 'http://www.wikidata.org/entity/Q53340'), ('Pname', 'Bob Bryan'), ('countWin', '14')]
[('player', 'http://www.wikidata.org/entity/Q53342'), ('Pname', 'Mike Bryan'), ('countWin', '11')]
[('player', 'http://www.wikidata.org/entity/Q229128'), ('Pname', 'Cara Black'), ('countWin', '10')]
[('player', 'http://www.wikidata.org/entity/Q54537'), ('Pname', 'Bethanie Mattek

## Task 7
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who won at least one edition of each Grand Slam (the result set must be a list of country IRI and label).

In [11]:
# Let's find the so called "country for sport property" for tennis players

queryString = """
SELECT DISTINCT ?p ?Pname 
WHERE {
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           ?p ?obj.
           
    ?p sc:name ?Pname.
    
    FILTER(REGEX(?Pname, "[Cc]ountry"))

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('Pname', 'country for sport')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('Pname', 'country of citizenship')]
2


In [12]:
#[('p', 'http://www.wikidata.org/prop/direct/P1532'), ('Pname', 'country for sport')]
#Let's check if it returns well the countries

queryString = """
SELECT DISTINCT ?country ?Cname
WHERE {
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P1532 ?country.       #countryForSport
           
    ?country sc:name ?Cname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q216923'), ('Cname', 'Chinese Taipei')]
[('country', 'http://www.wikidata.org/entity/Q1019'), ('Cname', 'Madagascar')]
[('country', 'http://www.wikidata.org/entity/Q129286'), ('Cname', 'British India')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('Cname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('Cname', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('Cname', "People's Republic of China")]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('Cname', 'Soviet Union')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('Cname', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('Cname', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('Cname', 'Canada')]
10


In [13]:
#Let's return players (and countries) who won one slam (not each one)

queryString = """
SELECT DISTINCT ?Pname ?country ?Cname
WHERE {
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P1532 ?country.       #countryForSport
           
           
    ?granSlam wdt:P361 wd:Q102113. #partOf GranSlams
    ?tournament wdt:P31 ?granSlam; #a
                wdt:P527 ?parts. #hasPart
                
    ?parts wdt:P1346 ?player. #winner
    
    ?player sc:name ?Pname.
    ?country sc:name ?Cname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('Pname', 'Eileen Bennett Whittingstall'), ('country', 'http://www.wikidata.org/entity/Q145'), ('Cname', 'United Kingdom')]
[('Pname', 'Markéta Vondroušová'), ('country', 'http://www.wikidata.org/entity/Q213'), ('Cname', 'Czech Republic')]
[('Pname', 'Stan Wawrinka'), ('country', 'http://www.wikidata.org/entity/Q39'), ('Cname', 'Switzerland')]
[('Pname', 'Nancy Richey'), ('country', 'http://www.wikidata.org/entity/Q30'), ('Cname', 'United States of America')]
[('Pname', 'Borna Ćorić'), ('country', 'http://www.wikidata.org/entity/Q224'), ('Cname', 'Croatia')]
[('Pname', 'Agnieszka Radwańska'), ('country', 'http://www.wikidata.org/entity/Q36'), ('Cname', 'Poland')]
[('Pname', 'Shirley Brasher'), ('country', 'http://www.wikidata.org/entity/Q145'), ('Cname', 'United Kingdom')]
[('Pname', 'Marcelo Melo'), ('country', 'http://www.wikidata.org/entity/Q155'), ('Cname', 'Brazil')]
[('Pname', 'Manuel Santana'), ('country', 'http://www.wikidata.org/entity/Q29'), ('Cname', 'Spain')]
[('Pn

In [22]:
#We have the bgp for each gran slam, my first guess is to make an intersection thanks to the word VALUES by filtering distinct counts = 4
#Let's first try to access winners of any subtournament of the French Open

queryString = """
SELECT DISTINCT ?Pname ?country ?Cname
WHERE {
   
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P1532 ?country.       #countryForSport
           
    ?yearlyFrenchOpen wdt:P31 wd:Q43605; #instanceOf FrenchOpen
                      wdt:P527 ?parts. #hasPart
                      
    ?parts wdt:P1346 ?player. #winner
    
    ?player sc:name ?Pname.
    ?country sc:name ?Cname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('Pname', 'Eileen Bennett Whittingstall'), ('country', 'http://www.wikidata.org/entity/Q145'), ('Cname', 'United Kingdom')]
[('Pname', 'Stan Wawrinka'), ('country', 'http://www.wikidata.org/entity/Q39'), ('Cname', 'Switzerland')]
[('Pname', 'Nancy Richey'), ('country', 'http://www.wikidata.org/entity/Q30'), ('Cname', 'United States of America')]
[('Pname', 'Agnieszka Radwańska'), ('country', 'http://www.wikidata.org/entity/Q36'), ('Cname', 'Poland')]
[('Pname', 'Shirley Brasher'), ('country', 'http://www.wikidata.org/entity/Q145'), ('Cname', 'United Kingdom')]
[('Pname', 'Marcelo Melo'), ('country', 'http://www.wikidata.org/entity/Q155'), ('Cname', 'Brazil')]
[('Pname', 'Manuel Santana'), ('country', 'http://www.wikidata.org/entity/Q29'), ('Cname', 'Spain')]
[('Pname', 'Anders Järryd'), ('country', 'http://www.wikidata.org/entity/Q34'), ('Cname', 'Sweden')]
[('Pname', 'Rika Hiraki'), ('country', 'http://www.wikidata.org/entity/Q17'), ('Cname', 'Japan')]
[('Pname', 'Victoria Az

In [25]:
#Let's see if the keyword values work : (note we are doing an union here, not intersection)

#[('slam', 'http://www.wikidata.org/entity/Q43605'), ('slamName', 'French Open')]
#[('slam', 'http://www.wikidata.org/entity/Q123577'), ('slamName', 'US Open')]
#[('slam', 'http://www.wikidata.org/entity/Q41520'), ('slamName', 'Wimbledon Championships')]
#[('slam', 'http://www.wikidata.org/entity/Q60874'), ('slamName', 'Australian Open')]


queryString = """
SELECT DISTINCT ?Pname ?country ?Cname
WHERE {
   
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P1532 ?country.       #countryForSport
    
    VALUES ?GrandSlam { wd:Q43605 wd:Q123577 wd:Q41520 wd:Q60874 } #FrenchOpen USOpen Wimbledon AustralianOpen

    ?yearlyGrandSlam wdt:P31 ?GrandSlam; #instanceOf 
                      wdt:P527 ?parts. #hasPart
                      
    ?parts wdt:P1346 ?player. #winner
    
    ?player sc:name ?Pname.
    ?country sc:name ?Cname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('Pname', 'Eileen Bennett Whittingstall'), ('country', 'http://www.wikidata.org/entity/Q145'), ('Cname', 'United Kingdom')]
[('Pname', 'Markéta Vondroušová'), ('country', 'http://www.wikidata.org/entity/Q213'), ('Cname', 'Czech Republic')]
[('Pname', 'Stan Wawrinka'), ('country', 'http://www.wikidata.org/entity/Q39'), ('Cname', 'Switzerland')]
[('Pname', 'Nancy Richey'), ('country', 'http://www.wikidata.org/entity/Q30'), ('Cname', 'United States of America')]
[('Pname', 'Borna Ćorić'), ('country', 'http://www.wikidata.org/entity/Q224'), ('Cname', 'Croatia')]
[('Pname', 'Agnieszka Radwańska'), ('country', 'http://www.wikidata.org/entity/Q36'), ('Cname', 'Poland')]
[('Pname', 'Shirley Brasher'), ('country', 'http://www.wikidata.org/entity/Q145'), ('Cname', 'United Kingdom')]
[('Pname', 'Marcelo Melo'), ('country', 'http://www.wikidata.org/entity/Q155'), ('Cname', 'Brazil')]
[('Pname', 'Manuel Santana'), ('country', 'http://www.wikidata.org/entity/Q29'), ('Cname', 'Spain')]
[('Pn

In [33]:
#Let's try now the intersection, by filtering the count of distinct slams found equal to the number of different values : 4

queryString = """
SELECT DISTINCT ?country ?Cname
WHERE {
   
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P1532 ?country.       #countryForSport
    
    VALUES ?GrandSlam { wd:Q43605 wd:Q123577 wd:Q41520 wd:Q60874 } #FrenchOpen USOpen Wimbledon AustralianOpen

    ?yearlyGrandSlam wdt:P31 ?GrandSlam; #instanceOf 
                      wdt:P527 ?parts. #hasPart
                      
    ?parts wdt:P1346 ?player. #winner
    
    ?player sc:name ?Pname.
    ?country sc:name ?Cname.
}
GROUP BY ?country ?Cname 
HAVING (COUNT(DISTINCT ?GrandSlam) = 4) 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q159'), ('Cname', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('Cname', "People's Republic of China")]
[('country', 'http://www.wikidata.org/entity/Q29'), ('Cname', 'Spain')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('Cname', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('Cname', 'Switzerland')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('Cname', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('Cname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('Cname', 'India')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('Cname', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('Cname', 'Kingdom of the Netherlands')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('Cname', 'Poland')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('Cname', 'Latvia')]
[('country', 'http://www.wikidata.org/entity/Q33946'), ('Cname'

Final query for this task

In [2]:
# Final Query

queryString = """
SELECT DISTINCT ?country ?Cname
WHERE {
   
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P1532 ?country.       #countryForSport
    
    VALUES ?GrandSlam { wd:Q43605 wd:Q123577 wd:Q41520 wd:Q60874 } #FrenchOpen USOpen Wimbledon AustralianOpen

    ?yearlyGrandSlam wdt:P31 ?GrandSlam; #instanceOf 
                      wdt:P527 ?parts. #hasPart
                      
    ?parts wdt:P1346 ?player. #winner
    
    ?player sc:name ?Pname.
    ?country sc:name ?Cname.
}
GROUP BY ?country ?Cname 
HAVING (COUNT(DISTINCT ?GrandSlam) = 4) 
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q159'), ('Cname', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('Cname', "People's Republic of China")]
[('country', 'http://www.wikidata.org/entity/Q29'), ('Cname', 'Spain')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('Cname', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q39'), ('Cname', 'Switzerland')]
[('country', 'http://www.wikidata.org/entity/Q31'), ('Cname', 'Belgium')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('Cname', 'France')]
[('country', 'http://www.wikidata.org/entity/Q668'), ('Cname', 'India')]
[('country', 'http://www.wikidata.org/entity/Q38'), ('Cname', 'Italy')]
[('country', 'http://www.wikidata.org/entity/Q29999'), ('Cname', 'Kingdom of the Netherlands')]
[('country', 'http://www.wikidata.org/entity/Q36'), ('Cname', 'Poland')]
[('country', 'http://www.wikidata.org/entity/Q211'), ('Cname', 'Latvia')]
[('country', 'http://www.wikidata.org/entity/Q33946'), ('Cname'

## Task 8
Consider the country for sport property of the tennis player (NO Country of citizenship). Return the countries with at least one player who was ranked 1 in the tennis ranking  (the result set must be a list of country IRI and label).

In [8]:
# We have all bgp required except the "ranking" one, let's look for it.
# We can start seeing properties related to Roger Federer 

queryString = """
SELECT DISTINCT ?p ?Pname 
WHERE {
   wd:Q1426 ?p ?obj . #Federer
   ?p sc:name ?Pname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('Pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('Pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('Pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('Pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('Pname', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('Pname', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('Pname', 'Munzinger person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('Pname', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('Pname', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('Pname', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1352')

In [14]:
#[('p', 'http://www.wikidata.org/prop/direct/P1352'), ('Pname', 'ranking')]
#Let's check its format

queryString = """
SELECT DISTINCT ?ranking
WHERE {
   wd:Q1426 wdt:P1352 ?ranking. #Federer ranking
   }
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('ranking', '1')]
[('ranking', '24')]
2


In [15]:
#So rankings are a list of values, let's see if the filtering players that have been once 1 works (like Federer who has 2 rankings)

queryString = """
SELECT DISTINCT ?ranking
WHERE {
   wd:Q1426 wdt:P1352 ?ranking. #Federer ranking
    FILTER(?ranking = 1 )
   }
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('ranking', '1')]
1


In [3]:
#Now let's see if we can access the rank of all tennis players 

queryString = """
SELECT DISTINCT ?PlayerName ?ranking
WHERE {
   
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P1352 ?ranking.
           
    ?player sc:name ?PlayerName
   }
   
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('PlayerName', 'Anna Karolína Schmiedlová'), ('ranking', '26')]
[('PlayerName', 'Wesley Koolhof'), ('ranking', '5')]
[('PlayerName', 'Wesley Koolhof'), ('ranking', '462')]
[('PlayerName', 'Rika Fujiwara'), ('ranking', '13')]
[('PlayerName', 'Rika Fujiwara'), ('ranking', '84')]
[('PlayerName', 'Margarita Gasparyan'), ('ranking', '25')]
[('PlayerName', 'Radu Albot'), ('ranking', '56')]
[('PlayerName', 'Radu Albot'), ('ranking', '108')]
[('PlayerName', 'Markéta Vondroušová'), ('ranking', '14')]
[('PlayerName', 'Kim Jones'), ('ranking', '25')]
[('PlayerName', 'Sviatlana Pirazhenka'), ('ranking', '371')]
[('PlayerName', 'Stan Wawrinka'), ('ranking', '3')]
[('PlayerName', 'Stan Wawrinka'), ('ranking', '19')]
[('PlayerName', 'Stan Wawrinka'), ('ranking', '88')]
[('PlayerName', 'Stan Wawrinka'), ('ranking', '287')]
[('PlayerName', 'Sabine Lisicki'), ('ranking', '12')]
[('PlayerName', 'Sabine Lisicki'), ('ranking', '35')]
[('PlayerName', 'Sabine Lisicki'), ('ranking', '92')]
[('PlayerN

In [8]:
#Let's try now to filter by those who have been 1.

queryString = """
SELECT DISTINCT ?PlayerName ?ranking
WHERE {
   
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P1352 ?ranking.
           
    ?player sc:name ?PlayerName.
    FILTER(?ranking = 1).
   }
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('PlayerName', 'Agnieszka Radwańska'), ('ranking', '1')]
[('PlayerName', 'Anders Järryd'), ('ranking', '1')]
[('PlayerName', 'Tom Okker'), ('ranking', '1')]
[('PlayerName', 'Nenad Zimonjić'), ('ranking', '1')]
[('PlayerName', 'Jimmy Connors'), ('ranking', '1')]
[('PlayerName', 'Victoria Azarenka'), ('ranking', '1')]
[('PlayerName', 'Henri Kontinen'), ('ranking', '1')]
[('PlayerName', 'Robert Seguso'), ('ranking', '1')]
[('PlayerName', 'Stefan Edberg'), ('ranking', '1')]
[('PlayerName', 'Hsieh Su-wei'), ('ranking', '1')]
10


In [11]:
#We can now build the answer in the required format :

queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P1352 ?ranking;       #ranking
           wdt:P1532 ?country.       #countryForSport   
           
    ?country sc:name ?countryName.
    FILTER(?ranking = 1).
   }
   
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q216923'), ('countryName', 'Chinese Taipei')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('countryName', "People's Republic of China")]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('countryName', 'Soviet Union')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryName', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryName', 'Canada')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryName', 'Japan')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('countryName', 'Belarus')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('countryName

Final query for this task

In [12]:
# Final query

queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
   
   ?player wdt:P31 wd:Q5;            #a Human
           wdt:P106 wd:Q10833314;    #profession TennisPlayer
           wdt:P1352 ?ranking;       #ranking
           wdt:P1532 ?country.       #countryForSport   
           
    ?country sc:name ?countryName.
    FILTER(?ranking = 1).
   }
   
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q216923'), ('countryName', 'Chinese Taipei')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom')]
[('country', 'http://www.wikidata.org/entity/Q148'), ('countryName', "People's Republic of China")]
[('country', 'http://www.wikidata.org/entity/Q15180'), ('countryName', 'Soviet Union')]
[('country', 'http://www.wikidata.org/entity/Q155'), ('countryName', 'Brazil')]
[('country', 'http://www.wikidata.org/entity/Q159'), ('countryName', 'Russia')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('countryName', 'Canada')]
[('country', 'http://www.wikidata.org/entity/Q17'), ('countryName', 'Japan')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q184'), ('countryName', 'Belarus')]
[('country', 'http://www.wikidata.org/entity/Q191'), ('countryName